In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1090,2024-08-04,Canadá Cebl,21:00,Edmonton Stingers,Calgary Surge,1.73,1.94,174.5,1.83,1.83,0.0,0.00,0.00,bPgdgUAk,0.578035,0.515464,0.546448,0.546448,0.093499,152.262,24.551114,0.161243,2.4,1.341641,0.559017,149.24,1.650,0.218289,0.132296,11.0,162.036,33.823100,0.208738,1.8,1.643168,0.912871,119.28,1.732,0.219249,0.126587,28.0,91,84,1.64,1.42,0.000,0.000,0.080922,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1091,2024-08-04,Vietnã Vba,09:30,Da Nang Dragons,Nhatrang Dolphins,3.19,1.32,168.5,1.83,1.83,0.0,0.00,0.00,lW1bodQk,0.313480,0.757576,0.546448,0.546448,0.071055,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,167.134,95.215204,0.569694,2.4,1.341641,0.559017,130.20,1.772,0.716254,0.404207,62.0,84,84,5.77,1.55,0.000,0.000,0.586381,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1092,2024-08-04,Mundo Jogos Olímpicos,12:15,Alemanha F,EUA F,14.10,1.01,162.5,1.89,1.89,0.0,0.00,0.00,zBSF1LFk,0.070922,0.990099,0.529101,0.529101,0.061021,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,52,101,3.97,1.01,0.000,0.000,1.225153,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1093,2024-08-04,Canadá Cebl,18:30,Niagara River,Ottawa Blackjacks,1.41,2.62,180.5,1.83,1.83,0.0,0.00,0.00,pt0Z0LCb,0.709220,0.381679,0.546448,0.546448,0.090899,132.938,18.480011,0.139012,1.8,1.643168,0.912871,111.20,1.578,0.242837,0.153889,-20.0,204.252,33.719403,0.165087,1.8,1.643168,0.912871,222.30,2.250,0.252587,0.112261,29.0,80,90,1.39,2.47,0.000,0.000,0.424615,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1094,2024-08-04,Chile Lnb 2,19:00,Stadio Italiano,Truenos De Talca,2.06,1.64,137.5,1.80,1.86,0.0,0.00,0.00,WSYkDUHM,0.485437,0.609756,0.555556,0.537634,0.095193,160.070,76.117629,0.475527,1.8,1.643168,0.912871,182.90,2.564,1.400493,0.546214,2.0,93.008,24.978857,0.268567,2.4,1.341641,0.559017,63.84,1.286,0.197307,0.153427,39.0,62,48,2.95,1.33,118.242,111.490,0.160532,0.023184,NaN,1.75,0.350,3.028571,0.000000,0.0,0.000000,0.14,0.028,22.857143,0.000000,0.0,0.000000
1095,2024-08-04,México Lnbp,21:00,Halcones Rojos,Correcaminos,1.41,2.62,174.5,1.83,1.83,0.0,0.00,0.00,tjrySCTO,0.709220,0.381679,0.546448,0.546448,0.090899,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,74,95,1.25,3.69,0.000,0.000,0.424615,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1096,2024-08-04,México Lnbp,19:00,Lobos Plateados,Freseros,1.35,2.86,177.5,1.83,1.83,0.0,0.00,0.00,dxkmwwxQ,0.740741,0.349650,0.546448,0.546448,0.090391,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,92,94,1.29,3.37,0.000,0.000,0.507236,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1097,2024-08-04,México Lnbp,19:15,Fuerza Regia,Plateros,1.22,3.64,155.5,1.81,1.85,0.0,0.00,0.00,h8zJXGam,0.819672,0.274725,0.552486,0.540541,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1098,21:00,México Lnbp,Abejas,Soles,1.59,Back Home
